In [7]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pints
import pints.toy as toy
import matplotlib.pyplot as plt
import seaborn as sns
from multiprocessing import Pool
from PFClasses import SimpleModel, ParticleFilter, Kalman, Event, StochasticModelTauLeaping
plt.rcParams['figure.figsize'] = (17, 7)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [148]:
class myPintsLikelihood(pints.ProblemLogLikelihood):
    def __init__(self, measurements, N):
        self.measurements = measurements
        self.x0 = measurements[0]
        self._n_parameters = 3
        self.N = N
    
    def __call__(self, x):
        b, k, r = x
        
        birth = Event(
            rate_calculator=lambda x: max(0, x[0] * b * (1 - float(x[0])/k)),
            change_function=lambda x: [x[0] + 1]
            )
        
        model = StochasticModelTauLeaping(x_0=np.array([100.]), events=[birth], step_size=1, R=np.array([[r]]))
        
        pf = ParticleFilter(model=model, N=self.N, measurements=self.measurements,
                           prior_cov_matrix=np.array([[50.]]))
        
        return pf.run()[2]

In [145]:
b = .01
K = 1000
sigma = 50
x_0 = np.array([100.])
times = range(10)

real_birth = Event(
    rate_calculator=lambda x: max(0, x[0] * b * (1 - float(x[0])/K)),
    change_function=lambda x: [x[0] + 1]
    )
real_model = StochasticModelTauLeaping(x_0, events=[real_birth], step_size=1, 
                                   R=np.array([[sigma]]))

real_measurements = [real_model.measure(t) for t in times]

In [149]:
import time

In [154]:
def test(n, t):
    b = .01
    K = 1000
    sigma = 50
    x_0 = np.array([100.])
    times = range(t)

    real_birth = Event(
        rate_calculator=lambda x: max(0, x[0] * b * (1 - float(x[0])/K)),
        change_function=lambda x: [x[0] + 1]
        )
    real_model = StochasticModelTauLeaping(x_0, events=[real_birth], step_size=1, 
                                       R=np.array([[sigma]]))

    real_measurements = [real_model.measure(t) for t in times]
    
    loglikelihood = myPintsLikelihood(real_measurements, n)
    y0 = np.array([.0025, 800, 45])
    boundaries_3d = pints.RectangularBoundaries([0, 500, 30], [.05, 1200, 55])

    opt = pints.OptimisationController(loglikelihood, y0, boundaries=boundaries_3d, method=pints.XNES)
    opt.set_log_to_screen(False)
    
    start = time.time()
    y1, g1 = opt.run()
    end = time.time()
    
    print(f"n={n}, t={t} -> {end-start}s")

In [157]:
for n in [1, 3, 5, 10]:
    for t in [10, 30, 50, 100]:
        test(n, t)

n=1, t=10 -> 4.419275760650635s
n=1, t=30 -> 15.867534875869751s
n=1, t=50 -> 62.929118156433105s
n=1, t=100 -> 65.47749400138855s
n=3, t=10 -> 13.753007173538208s
n=3, t=30 -> 31.64974069595337s
n=3, t=50 -> 84.65281200408936s
n=3, t=100 -> 202.90730118751526s
n=5, t=10 -> 10.278412818908691s
n=5, t=30 -> 34.54434323310852s
n=5, t=50 -> 134.88819289207458s
n=5, t=100 -> 134.79483914375305s
n=10, t=10 -> 51.29954814910889s
n=10, t=30 -> 86.66176009178162s
n=10, t=50 -> 174.16508102416992s
n=10, t=100 -> 207.8514528274536s


# 5 particles, 100 steps

In [146]:
loglikelihood = myPintsLikelihood(real_measurements)
y0 = np.array([.0025, 800, 45])
boundaries_3d = pints.RectangularBoundaries([0, 500, 30], [.05, 1200, 55])

opt = pints.OptimisationController(loglikelihood, y0, boundaries=boundaries_3d, method=pints.XNES)
opt.set_log_to_screen(False)
y1, g1 = opt.run()
print('Estimated parameters:')
print(y1)

# 1 particle, 30 steps

In [124]:
loglikelihood = myPintsLikelihood(real_measurements)

In [125]:
y0 = np.array([.0025, 800, 45])
boundaries_3d = pints.RectangularBoundaries([0, 500, 30], [.05, 1200, 55])

opt = pints.OptimisationController(loglikelihood, y0, boundaries=boundaries_3d, method=pints.XNES)
opt.set_log_to_screen(False)
y1, g1 = opt.run()
print('Estimated parameters:')
print(y1)

Estimated parameters:
[1.31338748e-02 1.07759008e+03 5.30010463e+01]


# 1 particle, 100 steps

In [110]:
loglikelihood = myPintsLikelihood(real_measurements)
y0 = np.array([.0025, 800, 45])
boundaries_3d = pints.RectangularBoundaries([0, 500, 30], [.05, 1200, 55])

opt = pints.OptimisationController(loglikelihood, y0, boundaries=boundaries_3d, method=pints.XNES)
opt.set_log_to_screen(False)
y1, g1 = opt.run()
print('Estimated parameters:')
print(y1)

Estimated parameters:
[9.13291499e-03 1.12138499e+03 4.91663485e+01]


# 5 particles, 30 steps

In [105]:
loglikelihood = myPintsLikelihood(real_measurements)
y0 = np.array([.0025, 800, 45])
boundaries_3d = pints.RectangularBoundaries([0, 500, 30], [.05, 1200, 55])

opt = pints.OptimisationController(loglikelihood, y0, boundaries=boundaries_3d, method=pints.XNES)
opt.set_log_to_screen(False)
y1, g1 = opt.run()
print('Estimated parameters:')
print(y1)

Estimated parameters:
[1.04893354e-02 9.63881813e+02 5.48382507e+01]


# 10 particles, 30 steps

In [107]:
loglikelihood = myPintsLikelihood(real_measurements)
y0 = np.array([.0025, 800, 45])
boundaries_3d = pints.RectangularBoundaries([0, 500, 30], [.05, 1200, 55])

opt = pints.OptimisationController(loglikelihood, y0, boundaries=boundaries_3d, method=pints.XNES)
opt.set_log_to_screen(False)
y1, g1 = opt.run()
print('Estimated parameters:')
print(y1)

Estimated parameters:
[1.16765204e-02 9.55850060e+02 5.45751339e+01]


# 3 particles, 70 steps

In [113]:
loglikelihood = myPintsLikelihood(real_measurements)
y0 = np.array([.0025, 800, 45])
boundaries_3d = pints.RectangularBoundaries([0, 500, 30], [.05, 1200, 55])

opt = pints.OptimisationController(loglikelihood, y0, boundaries=boundaries_3d, method=pints.XNES)
opt.set_log_to_screen(False)
y1, g1 = opt.run()
print('Estimated parameters:')
print(y1)

Estimated parameters:
[8.58496630e-03 7.91919106e+02 4.84775920e+01]


# 3 particles, 100 steps

In [115]:
loglikelihood = myPintsLikelihood(real_measurements)
y0 = np.array([.0025, 800, 45])
boundaries_3d = pints.RectangularBoundaries([0, 500, 30], [.05, 1200, 55])

opt = pints.OptimisationController(loglikelihood, y0, boundaries=boundaries_3d, method=pints.XNES)
opt.set_log_to_screen(False)
y1, g1 = opt.run()
print('Estimated parameters:')
print(y1)

Estimated parameters:
[1.00655340e-02 9.96240604e+02 4.71077753e+01]


# 1 particle, 70 steps

In [118]:
loglikelihood = myPintsLikelihood(real_measurements)
y0 = np.array([.0025, 800, 45])
boundaries_3d = pints.RectangularBoundaries([0, 500, 30], [.05, 1200, 55])

opt = pints.OptimisationController(loglikelihood, y0, boundaries=boundaries_3d, method=pints.XNES)
opt.set_log_to_screen(False)
y1, g1 = opt.run()
print('Estimated parameters:')
print(y1)

Estimated parameters:
[8.91025990e-03 7.74046291e+02 5.32435102e+01]


# 3, 30

In [121]:
loglikelihood = myPintsLikelihood(real_measurements)
y0 = np.array([.0025, 800, 45])
boundaries_3d = pints.RectangularBoundaries([0, 500, 30], [.05, 1200, 55])

opt = pints.OptimisationController(loglikelihood, y0, boundaries=boundaries_3d, method=pints.XNES)
opt.set_log_to_screen(False)
y1, g1 = opt.run()
print('Estimated parameters:')
print(y1)

Estimated parameters:
[1.06359634e-02 5.22929501e+02 3.27272124e+01]


# 5 70

In [128]:
loglikelihood = myPintsLikelihood(real_measurements)
y0 = np.array([.0025, 800, 45])
boundaries_3d = pints.RectangularBoundaries([0, 500, 30], [.05, 1200, 55])

opt = pints.OptimisationController(loglikelihood, y0, boundaries=boundaries_3d, method=pints.XNES)
opt.set_log_to_screen(False)
y1, g1 = opt.run()
print('Estimated parameters:')
print(y1)

Estimated parameters:
[1.18327354e-02 8.83117739e+02 4.22091793e+01]


# 10 70

In [131]:
loglikelihood = myPintsLikelihood(real_measurements)
y0 = np.array([.0025, 800, 45])
boundaries_3d = pints.RectangularBoundaries([0, 500, 30], [.05, 1200, 55])

opt = pints.OptimisationController(loglikelihood, y0, boundaries=boundaries_3d, method=pints.XNES)
opt.set_log_to_screen(False)
y1, g1 = opt.run()
print('Estimated parameters:')
print(y1)

Estimated parameters:
[9.00083923e-03 6.38142167e+02 4.85177839e+01]


In [53]:
def check(llfunction):
    y0 = np.array([.5, 200 ,1])
    boundaries_3d = pints.RectangularBoundaries([0, 5, 1e-3], [1, 500, 10])

    opt = pints.OptimisationController(llfunction, y0, boundaries=boundaries_3d, method=pints.XNES)
    opt.set_log_to_screen(False)
    y1, g1 = opt.run()
    print('Estimated parameters:')
    print(y1)